In [1]:
from sklearn.metrics import classification_report
from keras.layers import Dense, Flatten, Dropout, Input
from keras.models import Model
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import random




ModuleNotFoundError: No module named 'keras'

In [ ]:
!conda install tensorflow


In [ ]:
# In[48]:


img_rows = img_cols = 28
path_to_data = (r'C:\Users\cjabz\Downloads\mnist_bin')


def load_data(path_to_data, img_rows, img_cols):
    print('Загрузка данных из двоичных файлов...')
    with open(path_to_data + '\images_trn.bin', 'rb') as rb:
        x_trn = np.fromfile(rb, dtype = np.uint8)
    with open(path_to_data + '\labels_trn.bin', 'rb') as rb:
        y_trn = np.fromfile(rb, dtype = np.uint8)
    with open(path_to_data + '\images_tst.bin', 'rb') as rb:
        x_tst = np.fromfile(rb, dtype = np.uint8)
    with open(path_to_data + '\labels_tst.bin', 'rb') as rb:
        y_tst = np.fromfile(rb, dtype = np.uint8)
    x_trn = x_trn.reshape(-1, img_rows * img_cols)
    x_tst = x_tst.reshape(-1, img_rows * img_cols)
    return x_trn, y_trn, x_tst, y_tst

# Загрузка обучающего и проверочного множества из бинарных файлов
# Загружаются изображения и их метки
x_trn, y_trn, x_tst, y_tst = load_data(path_to_data, img_rows, img_cols)
print("End")




In [ ]:
# In[49]:


def plot_images(images, labels, num_images=10):
    plt.figure(figsize=(10, 10))
    n = random.sample(range(len(images)), 25)
    for i in range(0, num_images):
        ax = plt.subplot(5, 5, i+1)
        plt.imshow(images[n[i]].reshape(img_rows, img_cols), cmap='gray')
        plt.title(labels[n[i]])
        plt.axis('off')
    plt.show()

# Вывод первых 10 изображений из обучающего набора
plot_images(x_trn, y_trn, 25)





In [ ]:
# In[50]:


# Нормализация пиксельных значений
x_trn = x_trn / 255.0
x_tst = x_tst / 255.0

# Преобразование меток в one-hot векторы
# y_trn = to_categorical(y_trn, num_classes=10)
# y_tst = to_categorical(y_tst, num_classes=10)
print("Форма тренировочных данных:", y_trn.shape)
print("Форма тестовых данных:", y_tst.shape)

y_trn = tf.keras.utils.to_categorical(y_trn, 10)
y_tst = tf.keras.utils.to_categorical(y_tst, 10)

print("Форма тренировочных данных:", y_trn.shape)
print("Форма тестовых данных:", y_tst.shape)

# Создание модели
inp = Input((img_rows * img_cols, 1))
m = Flatten()(inp)
m = Dropout(0.3)(m)
m = Dense(units=128, activation="relu")(m)
m = Dense(units=64, activation="selu")(m)
m = Dense(units=32, activation="sigmoid")(m)
m = Dense(units=10, activation="softmax")(m)
model = Model(inputs=inp, outputs=m)
model.summary()

# Компиляция модели
model.compile(optimizer="Adam", loss="mse", metrics=["accuracy"])

# Обучение модели
h = model.fit(x_trn, y_trn, epochs=10, verbose=2, validation_data=(x_tst, y_tst))

# Сохранение модели
model.save('my_model')

# model.save(path_to_model)
# print_history(h.history)




In [ ]:
# In[51]:


!conda install h5py




In [ ]:
# In[52]:


# Вывод истории обучения
def one_plot(n, y_lb, loss_acc, val_loss_acc):
    plt.subplot(1, 2, n)
    if n == 1:
        lb, lb2 = 'loss', 'val_loss'
        yMin = 0
        yMax = 1.05 * max(max(loss_acc), max(val_loss_acc))
    else:
        lb, lb2 = 'acc', 'val_acc'
        yMin = min(min(loss_acc), min(val_loss_acc))
        yMax = 1.0
    plt.plot(loss_acc, color = 'r', label = lb, linestyle = '--')
    plt.plot(val_loss_acc, color = 'g', label = lb2)
    plt.ylabel(y_lb)
    plt.xlabel('Эпоха')
    plt.ylim([0.95 * yMin, yMax])
    plt.legend()

# История обучения; вывод графиков обучения
history = h.history
plt.figure(figsize = (9, 4))
plt.subplots_adjust(wspace = 0.5)
one_plot(1, 'Потери', history['loss'], history['val_loss'])
one_plot(2, 'Точность', history['accuracy'], history['val_accuracy'])
plt.suptitle('Потери и точность')
plt.show()




In [ ]:
# In[53]:


num_classes = 10
fn_model = 'my_model'
model = load_model(fn_model)

# Предсказание
y_pred_probs = model.predict(x_tst, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)  # Преобразование в вектор с метками классов

# Преобразование y_tst в вектор меток классов, чтобы они совпадали с форматами
y_tst_labels = np.argmax(y_tst, axis=1)

# Отчет классификации
print(classification_report(y_tst_labels, y_pred, digits=4, zero_division=0))

# Оценка модели
evl = model.evaluate(x_tst, y_tst, verbose=0)
print('Точность:', round(evl[1] * 100, 2))




In [ ]:
# In[54]:


def precision_recall_f1(y_true, y_pred, average='macro'):
    # Найти уникальные классы
    unique_classes = np.unique(np.concatenate((y_true, y_pred)))

    precisions = []
    recalls = []
    f1_scores = []

    for cls in unique_classes:
        # True Positives
        TP = np.sum((y_pred == cls) & (y_true == cls))
        # False Positives
        FP = np.sum((y_pred == cls) & (y_true != cls))
        # False Negatives
        FN = np.sum((y_pred != cls) & (y_true == cls))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    if average == 'macro':
        # Макроусреднение
        precision = np.mean(precisions)
        recall = np.mean(recalls)
        f1 = np.mean(f1_scores)
    elif average == 'micro':
        # Микроусреднение, суммируя все TP, FP и FN для всех классов
        TP = np.sum([(y_pred == cls) & (y_true == cls) for cls in unique_classes])
        FP = np.sum([(y_pred == cls) & (y_true != cls) for cls in unique_classes])
        FN = np.sum([(y_pred != cls) & (y_true == cls) for cls in unique_classes])

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = (2 * precision * recall) / (precision + recall)

    return precision, recall, f1




In [ ]:
# In[56]:


precision, recall, f1 = precision_recall_f1(y_tst_labels, y_pred, average='macro')
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")